In [1]:
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!git clone https://github.com/OscarOvanger/GeoDecepticon.git

Cloning into 'GeoDecepticon'...
remote: Enumerating objects: 19998, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 19998 (delta 139), reused 131 (delta 129), pack-reused 19846 (from 3)
Receiving objects: 100% (19998/19998), 106.92 MiB | 14.99 MiB/s, done.
Resolving deltas: 100% (417/417), done.
Updating files: 100% (78/78), done.


In [3]:
import os
os.chdir('/content/GeoDecepticon')

In [4]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import wandb
import os
from tqdm.notebook import tqdm
import copy

# Assume VisionTransformer class is defined elsewhere as in your updated model
from transformer_four import VisionTransformer, create_model

# Binary Image Dataset
class BinaryImageDataset(Dataset):
    def __init__(self, images):
        self.images = images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx]

# Function to visualize binary images
def plot_binary_image(image, title=None):
    plt.figure(figsize=(5, 5))
    plt.imshow(image.squeeze(), cmap='gray')
    if title:
        plt.title(title)
    plt.axis('off')
    return plt

# Training Pipeline
def train_vision_transformer(
    train_dataset,
    val_dataset=None,
    d_model=512,
    num_heads=8,
    num_layers=6,
    feedforward_dim=2048,
    batch_size=32,
    learning_rate=1e-4,
    num_epochs=1000,
    save_dir="checkpoints",
    condition_indices=None,
    condition_values=None,
    device=None
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {device}")

    # Create directories
    os.makedirs(save_dir, exist_ok=True)

    # Create DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    if val_dataset:
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Create model
    model = create_model(
        train_dataset,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers,
        feedforward_dim=feedforward_dim
    ).to(device)

    # Initialize wandb
    wandb.init(
        project="vision-transformer-4x4",
        config={
            "batch_size": batch_size,
            "d_model": d_model,
            "num_heads": num_heads,
            "feedforward_dim": feedforward_dim,
            "num_layers": num_layers,
            "num_tokens": model.num_tokens,
            "num_observed_tokens": model.num_observed_tokens,
            "dropout": 0.1,
            "learning_rate": learning_rate,
            "num_epochs": num_epochs,
        },
    )

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Track best model
    best_loss = float('inf')
    best_model_state = None

    # Masking rate scheduler - linear increase from 1 to max over epochs
    max_masks = 256  # All patches for a 64x64 image with 4x4 patches

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        # Calculate current masking rate (linear schedule from 1 to max_masks)
        current_max_masks = min(1 + int((max_masks - 1) * epoch / (num_epochs * 0.8)), max_masks)

        # For visualization
        first_batch_imgs = []
        first_batch_masks = []
        first_batch_reconstructions = []

        # Training loop
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, images in progress_bar:
            batch_size = images.shape[0]

            # Tokenize each image - convert to patches and map to token indices
            # Shape: (batch_size, 256) for 64x64 images with 4x4 patches
            token_indices = torch.stack([
                model.tokenize_image(img) for img in images
            ]).to(device)

            # Create random mask for each image in batch
            # Randomly select number of patches to mask for each image (between 1 and current_max_masks)
            num_masks_per_image = torch.randint(
                1, current_max_masks + 1, (batch_size,)
            ).tolist()

            # Prepare masked tokens and mask tensor
            masked_tokens = token_indices.clone()
            mask = torch.zeros_like(token_indices, dtype=torch.bool)

            # Create masks and apply partial masking for 30% of masked patches
            for b in range(batch_size):
                # Select random positions to mask
                positions_to_mask = torch.randperm(256)[:num_masks_per_image[b]]

                # Determine which positions get partial masks (30%)
                num_partial = int(0.3 * len(positions_to_mask))
                partial_positions = positions_to_mask[:num_partial]
                full_positions = positions_to_mask[num_partial:]

                # Apply full masking (replace with mask token)
                mask[b, full_positions] = True
                masked_tokens[b, full_positions] = model.mask_token_id

                # Apply partial masking (replace with partial mask tokens)
                for i, pos in enumerate(partial_positions):
                    mask[b, pos] = True

                    # Get original token and extract one observed cell
                    orig_token = token_indices[b, pos].item()
                    if orig_token < model.num_observed_tokens:  # Only for non-special tokens
                        # Get the original patch
                        if orig_token in model.token_to_patch:
                            orig_patch = model.token_to_patch[orig_token]

                            # Select a random position to observe (0-15 for 4x4 patch)
                            observed_pos = torch.randint(0, 16, (1,)).item()
                            observed_val = orig_patch[observed_pos].item()

                            # Calculate partial mask token id
                            partial_idx = observed_pos * 2 + int(observed_val)

                            # Assign partial mask token if available
                            if partial_idx < len(model.partial_mask_token_ids):
                                masked_tokens[b, pos] = model.partial_mask_token_ids[partial_idx]
                            else:
                                # Fallback to full mask if no suitable partial token
                                masked_tokens[b, pos] = model.mask_token_id
                        else:
                            masked_tokens[b, pos] = model.mask_token_id
                    else:
                        masked_tokens[b, pos] = model.mask_token_id

            # Forward pass
            logits = model(masked_tokens)

            # Extract only the masked positions for loss calculation
            masked_positions = mask.nonzero(as_tuple=True)
            masked_logits = logits[masked_positions[0], masked_positions[1]]
            masked_targets = token_indices[masked_positions[0], masked_positions[1]]

            # For partial masks, adjust target probabilities to include only compatible tokens
            # For each masked position, we need to filter logits if it's a partial mask
            filtered_logits = []
            filtered_targets = []

            for i in range(len(masked_positions[0])):
                b, pos = masked_positions[0][i], masked_positions[1][i]
                token_id = masked_tokens[b, pos].item()
                target = masked_targets[i].item()

                # If it's a partial mask token
                if token_id in model.partial_mask_token_ids:
                    # Determine which position and value are observed
                    partial_idx = model.partial_mask_token_ids.index(token_id)
                    observed_pos = partial_idx // 2
                    observed_val = partial_idx % 2

                    # Create a mask for compatible tokens (those with the same observed value)
                    compatible_mask = torch.zeros(model.num_tokens)

                    # Find all tokens in vocabulary that have the observed value at observed_pos
                    for t_id, patch in model.token_to_patch.items():
                        if t_id < model.num_observed_tokens and patch[observed_pos].item() == observed_val:
                            compatible_mask[t_id] = 1.0

                    # Zero out logits for incompatible tokens
                    current_logits = logits[b, pos].clone()
                    current_logits = current_logits * compatible_mask.to(device)

                    # Replace very negative values for zeros to maintain softmax stability
                    current_logits[compatible_mask.to(device) == 0] = -1e9

                    filtered_logits.append(current_logits)
                    filtered_targets.append(target)
                else:
                    # For full masks, use all logits
                    filtered_logits.append(logits[b, pos])
                    filtered_targets.append(target)

            # Stack filtered logits and targets
            if filtered_logits:
                filtered_logits = torch.stack(filtered_logits)
                filtered_targets = torch.tensor(filtered_targets).to(device)

                # Compute loss
                loss = criterion(filtered_logits, filtered_targets)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

                # Update progress bar
                progress_bar.set_postfix({"loss": loss.item(), "masks": f"{np.mean(num_masks_per_image):.1f}/{current_max_masks}"})
            else:
                # Skip if no masked positions (unlikely, but handle it)
                print("Warning: No masked positions in batch.")
                continue

            # Save first batch for visualization
            if batch_idx == 0:
                with torch.no_grad():
                    # Store original images
                    first_batch_imgs = images[:4].cpu()

                    # Store masked images
                    masked_images = []
                    for i in range(min(4, batch_size)):
                        masked_img = model.tokens_to_image(masked_tokens[i:i+1].cpu())
                        masked_images.append(masked_img.squeeze(0))
                    first_batch_masks = masked_images

                    # Generate reconstructions
                    reconstructed_tokens = masked_tokens[:4].cpu().clone()
                    mask_cpu = mask[:4].cpu()

                    # For each image in mini-batch
                    for i in range(min(4, batch_size)):
                        # Get positions of masked tokens
                        masked_positions_i = mask_cpu[i].nonzero(as_tuple=True)[0]

                        # Get predicted token ids
                        pred_logits = filtered_logits.reshape(-1, model.num_tokens)
                        pred_tokens = torch.argmax(pred_logits, dim=1).cpu()

                        # Count masked positions for this image
                        count = 0
                        for j, pos in enumerate(masked_positions):
                            if pos[0] == i and j < len(pred_tokens):
                                reconstructed_tokens[i, pos[1]] = pred_tokens[count]
                                count += 1

                    # Create reconstructed images
                    reconstructed_images = []
                    for i in range(min(4, batch_size)):
                        recon_img = model.tokens_to_image(reconstructed_tokens[i:i+1])
                        reconstructed_images.append(recon_img.squeeze(0))
                    first_batch_reconstructions = reconstructed_images

        # Calculate average loss for epoch
        avg_loss = total_loss / len(train_loader)

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_loss,
            "masked_patches": np.mean(num_masks_per_image),
            "max_masked_patches": current_max_masks,
        })

        # Visualization every 50 epochs or on the last epoch
        if (epoch + 1) % 50 == 0 or epoch == num_epochs - 1:
            # Log images to wandb
            if first_batch_imgs:
                fig, axes = plt.subplots(3, min(4, batch_size), figsize=(16, 9))
                for i in range(min(4, batch_size)):
                    # Original
                    axes[0, i].imshow(first_batch_imgs[i].squeeze(0), cmap='gray')
                    axes[0, i].set_title(f"Original {i+1}")
                    axes[0, i].axis('off')

                    # Masked
                    axes[1, i].imshow(first_batch_masks[i].squeeze(0), cmap='gray')
                    axes[1, i].set_title(f"Masked {i+1}")
                    axes[1, i].axis('off')

                    # Reconstructed
                    axes[2, i].imshow(first_batch_reconstructions[i].squeeze(0), cmap='gray')
                    axes[2, i].set_title(f"Reconstructed {i+1}")
                    axes[2, i].axis('off')

                plt.tight_layout()
                wandb.log({"sample_images": wandb.Image(plt)})
                plt.close()

        # Validation step
        if val_dataset:
            model.eval()
            val_loss = 0.0

            with torch.no_grad():
                for images in val_loader:
                    # Similar tokenization and masking logic as training
                    token_indices = torch.stack([
                        model.tokenize_image(img) for img in images
                    ]).to(device)

                    # Use fixed masking rate for validation
                    num_masks = min(64, current_max_masks)  # 25% of patches

                    # Prepare masked tokens and mask tensor
                    masked_tokens = token_indices.clone()
                    mask = torch.zeros_like(token_indices, dtype=torch.bool)

                    # Apply masking
                    for b in range(images.shape[0]):
                        positions_to_mask = torch.randperm(256)[:num_masks]
                        mask[b, positions_to_mask] = True
                        masked_tokens[b, positions_to_mask] = model.mask_token_id

                    # Forward pass
                    logits = model(masked_tokens)

                    # Calculate validation loss
                    masked_positions = mask.nonzero(as_tuple=True)
                    masked_logits = logits[masked_positions[0], masked_positions[1]]
                    masked_targets = token_indices[masked_positions[0], masked_positions[1]]

                    # Compute loss
                    batch_loss = criterion(masked_logits, masked_targets)
                    val_loss += batch_loss.item()

            # Calculate average validation loss
            avg_val_loss = val_loss / len(val_loader)
            wandb.log({"val_loss": avg_val_loss})

            # Save best model
            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                best_model_state = copy.deepcopy(model.state_dict())

                # Save checkpoint
                checkpoint_path = os.path.join(save_dir, f"best_model.pth")
                torch.save({
                    "epoch": epoch + 1,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "loss": avg_val_loss,
                }, checkpoint_path)

                wandb.save(checkpoint_path)
                print(f"New best model saved at epoch {epoch+1} with validation loss: {avg_val_loss:.4f}")

                # Generate samples with conditioning (requirement G)
                if condition_indices is not None and condition_values is not None:
                    generate_conditioned_samples(model, condition_indices, condition_values, device)
        else:
            # If no validation set, save based on training loss
            if avg_loss < best_loss:
                best_loss = avg_loss
                best_model_state = copy.deepcopy(model.state_dict())

                # Save checkpoint
                checkpoint_path = os.path.join(save_dir, f"best_model.pth")
                torch.save({
                    "epoch": epoch + 1,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "loss": avg_loss,
                }, checkpoint_path)

                wandb.save(checkpoint_path)
                print(f"New best model saved at epoch {epoch+1} with training loss: {avg_loss:.4f}")

                # Generate samples with conditioning (requirement G)
                if condition_indices is not None and condition_values is not None:
                    generate_conditioned_samples(model, condition_indices, condition_values, device)

        # Save checkpoint periodically
        if (epoch + 1) % 100 == 0:
            checkpoint_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch+1}.pth")
            torch.save({
                "epoch": epoch + 1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": avg_loss,
            }, checkpoint_path)
            wandb.save(checkpoint_path)

    # Finish wandb run
    wandb.finish()

    # Return best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    return model

def generate_conditioned_samples(model, condition_indices, condition_values, device):
    """Generate samples with fixed conditioning values at specified indices"""
    model.eval()

    # Convert to tensor
    condition_indices = torch.tensor(condition_indices, device=device)
    condition_values = torch.tensor(condition_values, device=device)

    # Generate multiple samples with the same conditioning
    num_samples = 3
    samples = []

    with torch.no_grad():
        for i in range(num_samples):
            # Start with all masked tokens
            tokens = torch.full((1, 256), model.mask_token_id, device=device)

            # Set the conditioning values
            for idx, val in zip(condition_indices, condition_values):
                # Convert flat index to 2D patch index
                img_size = 64
                patch_size = 4
                patches_per_row = img_size // patch_size

                pixel_x = idx % img_size
                pixel_y = idx // img_size

                patch_x = pixel_x // patch_size
                patch_y = pixel_y // patch_size

                patch_idx = patch_y * patches_per_row + patch_x

                # Set the conditioning value in the token
                if patch_idx < 256:  # Ensure it's a valid patch index
                    # We need to find a token that has the right value at the right position
                    within_patch_x = pixel_x % patch_size
                    within_patch_y = pixel_y % patch_size
                    within_patch_idx = within_patch_y * patch_size + within_patch_x

                    # Create a partial mask token or find a matching token
                    compatible_tokens = []

                    # Check if we have a partial mask token for this position and value
                    for partial_idx, token_id in enumerate(model.partial_mask_token_ids):
                        if partial_idx // 2 == within_patch_idx and partial_idx % 2 == val.item():
                            # Found a matching partial mask token
                            tokens[0, patch_idx] = token_id
                            break
                    else:
                        # If no partial mask token, find a token with matching value at position
                        for token_id, patch in model.token_to_patch.items():
                            if token_id < model.num_observed_tokens and patch[within_patch_idx].item() == val.item():
                                compatible_tokens.append(token_id)

                        if compatible_tokens:
                            # Randomly select a compatible token
                            tokens[0, patch_idx] = compatible_tokens[torch.randint(0, len(compatible_tokens), (1,)).item()]

            # Now autoregressively generate the rest
            image = model.generate_image(batch_size=1, temperature=0.8, device=device)
            samples.append(image.cpu())

    # Visualize and log the samples
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    for i in range(num_samples):
        axes[i].imshow(samples[i].squeeze(), cmap='gray')
        axes[i].set_title(f"Sample {i+1}")
        axes[i].axis('off')

    plt.tight_layout()
    wandb.log({"conditioned_samples": wandb.Image(plt)})
    plt.close()

    return samples

# Example usage in a notebook:

In [5]:
arr = np.load("Data/Markov_field/training_data.npz")["arr_0"]
training_data = arr[:9000]
test_data = arr[9000:]
training_data = training_data.reshape(-1, 64, 64)
val_data = test_data.reshape(-1, 64, 64)

In [ ]:
# Parameters
d_model = 512
num_heads = 8
num_layers = 6
feedforward_dim = 2048
batch_size = 32
learning_rate = 1e-4
num_epochs = 1000

# Load data (example)
train_images = torch.tensor(training_data, dtype=torch.float32)
val_images = torch.tensor(val_data, dtype=torch.float32)

# Create datasets
train_dataset = BinaryImageDataset(train_images)
val_dataset = BinaryImageDataset(val_images)

# Condition indices and values as specified
condition_indices = np.array([876,3825,2122,2892,1556,2683,3667,1767,483,2351,2000,3312,2953,289,2373,2720,872,2713,1206,1341,3541,2226,3423,1904,2882,2540,1497,2524,264,1441])
condition_values = np.array([0,1,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,1,0,1,0,1,1,0,1])

# Train model
model = train_vision_transformer(
    train_dataset,
    val_dataset,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    feedforward_dim=feedforward_dim,
    batch_size=batch_size,
    learning_rate=learning_rate,
    num_epochs=num_epochs,
    condition_indices=condition_indices,
    condition_values=condition_values
)

Using device: cuda
Building vocabulary from training data...
Found 30572 unique patches in training data
Total token vocabulary size: 30605


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: oscar-ovanger (oscars) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/1000:   0%|          | 0/282 [00:00<?, ?it/s]